In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
!pip install sweetviz
import sweetviz as sv
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.multioutput import MultiOutputClassifier

In [ ]:
test = pd.read_csv('../input/hackerearth-ml-challenge-pet-adoption/test.csv')
train = pd.read_csv('../input/hackerearth-ml-challenge-pet-adoption/train.csv')

In [ ]:
train['type'] = 0
test['breed_category'] = 0
test['pet_category'] = 0
test['type'] = 1

In [ ]:
train

In [ ]:
test

In [ ]:
master = pd.concat([train,test], axis = 0)
master

In [ ]:
master.isnull().sum()
master['condition'] = master['condition'].fillna(-1)

In [ ]:
master

In [ ]:
master['pet_id'] = master['pet_id'].str.findall(r'(ANSL_)(\d\d\d)(\d\d)')
master['pet_id'] = master['pet_id'].apply(lambda x: int(x[0][1]))
master

In [ ]:
def create_date_features(df,n,colDt,inclTime=False):
    cols_dat=[]

    df[str(n) + '_' + 'Year'] = pd.to_datetime(df[colDt]).dt.year
    cols_dat.append(str(n) + '_'+ 'Year')
    df[str(n) + '_' + 'Month'] = pd.to_datetime(df[colDt]).dt.month
    cols_dat.append(str(n) + '_'+ 'Month')
    df[str(n) + '_' + 'Day'] = pd.to_datetime(df[colDt]).dt.day
    cols_dat.append(str(n) + '_'+ 'Day')
    df[str(n) + '_' + 'Dayofweek'] = pd.to_datetime(df[colDt]).dt.dayofweek
    cols_dat.append(str(n) + '_'+ 'Dayofweek')
    df[str(n) + '_' + 'DayOfyear'] = pd.to_datetime(df[colDt]).dt.dayofyear
    cols_dat.append(str(n) + '_'+ 'DayOfyear')
    df[str(n) + '_' + 'Week'] = pd.to_datetime(df[colDt]).dt.week
    cols_dat.append(str(n) + '_'+ 'Week')
    df[str(n) + '_' + 'Quarter'] = pd.to_datetime(df[colDt]).dt.quarter 
    cols_dat.append(str(n) + '_'+ 'Quarter')
    df[str(n) + '_' + 'Is_month_start'] = pd.to_datetime(df[colDt]).dt.is_month_start*1
    cols_dat.append(str(n) + '_'+ 'Is_month_start')
    df[str(n) + '_' + 'Is_month_end'] = pd.to_datetime(df[colDt]).dt.is_month_end*1
    cols_dat.append(str(n) + '_'+ 'Is_month_end')
    df[str(n) + '_' + 'Is_quarter_start'] = pd.to_datetime(df[colDt]).dt.is_quarter_start*1
    cols_dat.append(str(n) + '_'+ 'Is_quarter_start')
    df[str(n) + '_' + 'Is_quarter_end'] = pd.to_datetime(df[colDt]).dt.is_quarter_end*1
    cols_dat.append(str(n) + '_'+ 'Is_quarter_end')
    df[str(n) + '_' + 'Is_year_start'] = pd.to_datetime(df[colDt]).dt.is_year_start*1
    cols_dat.append(str(n) + '_'+ 'Is_year_start')
    df[str(n) + '_' + 'Is_year_end'] = pd.to_datetime(df[colDt]).dt.is_year_end*1
    cols_dat.append(str(n) + '_'+ 'Is_year_end')
    df[str(n) + '_' + 'Semester'] = np.where(df[str(n) + '_' + 'Quarter'].isin([1,2]),1,2)
    cols_dat.append(str(n) + '_'+ 'Semester')
    df[str(n) + '_' + 'Is_weekend'] = np.where(df[str(n) + '_' + 'Dayofweek'].isin([5,6]),1,0)
    cols_dat.append(str(n) + '_'+ 'Is_weekend')
    df[str(n) + '_' + 'Is_weekday'] = np.where(df[str(n) + '_' + 'Dayofweek'].isin([0,1,2,3,4]),1,0)
    cols_dat.append(str(n) + '_'+ 'Is_weekday')
    df[str(n) + '_' + 'Days_in_month'] = pd.to_datetime(df[colDt]).dt.days_in_month
    if inclTime:
        df[str(n) + '_' + 'Hour'] = pd.to_datetime(df[colDt]).dt.hour
        cols_dat.append(str(n) + '_'+ 'Hour')
    #df[str(n) + '_' + 'Time'] = [((date.hour*60+(date.minute))*60)+date.second for date in df.DateTime]

    return df,cols_dat

In [ ]:
master2 = create_date_features(master,0,'issue_date',inclTime = False)[0]
master2 = create_date_features(master2,1,'listing_date',inclTime = True)[0]
master2

In [ ]:
sns.scatterplot(master2['color_type'],master2['breed_category'])

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
master2['color_type'] = LabelEncoder().fit_transform(master2['color_type'])

In [ ]:
master2.columns

In [ ]:
master2['wait'] = pd.to_datetime(master2['listing_date']) - pd.to_datetime(master2['issue_date'])
master2['wait'] = master2['wait'].apply(lambda x: pd.Timedelta(x).days)

In [ ]:
master2['area'] = master2['length(m)']*master2['height(cm)']
master2['area']

In [ ]:
master2['wait']

In [ ]:
master2 = master2.sort_values(by = 'listing_date')
lags = [1,2,3,4,5,6,7]
for lag in lags:
    master2['listing_breed_lag_'+str(lag)] = master2['breed_category'].shift(lag)
    master2['listing_pet_lag_'+str(lag)] = master2['pet_category'].shift(lag)

In [ ]:
master2 = master2.fillna(-1).reset_index()
master2 = master2.sort_values(by = ['type','index'])
master2 = master2.set_index('index')
master2

In [ ]:
my_report = sv.analyze(master)
my_report.show_html() 

In [ ]:
c = master2.corr().to_csv('corr.csv')

In [ ]:
cols_ignore_breed = ['issue_date', 'listing_date', 'type','pet_id','height(cm)','wait','listing_pet_lag_6','1_Is_quarter_start','0_Semester','1_Semester','0_Is_weekday','1_Is_year_start','1_Is_weekend','0_Is_year_start','0_Quarter',
                    '0_Is_month_start','1_Is_year_end','1_Is_month_start','1_Is_weekday']
cols_ignore_pet = ['issue_date', 'listing_date', 'type','1_Is_month_start','1_Is_year_end','0_Is_year_start','0_Day']
cols_y = ['breed_category','pet_category']
cols_X_breed = list(set(master2.columns)-set(cols_ignore_breed)-set(cols_y))
cols_X_pet = list(set(master2.columns)-set(cols_ignore_pet)-set(cols_y))

In [ ]:
train_new = master2[master2['type']==0]
test_new = master2[master2['type']==1]

In [ ]:
X_breed = train_new[cols_X_breed]
X_pet = train_new[cols_X_pet]
y_breed = train_new['breed_category']
y_pet = train_new['pet_category']

In [ ]:
cols_X_pet

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.4, random_state=0)

In [ ]:
X_test_breed = test_new[cols_X_breed]
X_test_pet = test_new[cols_X_pet]

In [ ]:
y_breed_test = []
for train_index, val_index in sss.split(X_breed, y_breed):
    print("TRAIN:", train_index, "val:", val_index)
    X_train, X_val = X_breed.loc[train_index,:], X_breed.loc[val_index,:]
    y_train, y_val = y_breed.loc[train_index,], y_breed.loc[val_index,]
    clf1 = lgb.LGBMClassifier(num_leaves = 31,learning_rate = 0.01,
                         n_estimators = 3000,objective = 'multiclass', num_class = 3,
                         reg_lambda = 3, silent = False).fit(X_train,np.array(y_train))
    y_preds = clf1.predict(X_val)
    y_breed_temp = clf1.predict(X_test_breed)
    y_breed_test.append(y_breed_temp)
    f1 = accuracy_score(y_val,y_preds)
    #lgb.plot_importance(clf,figsize = (20,20))
    print(f1)
print(y_breed_test)

In [ ]:
lgb.plot_importance(clf1,figsize = (20,20))

In [ ]:
df_breed_test = pd.DataFrame(y_breed_test)
df_breed_test = df_breed_test.T
df_breed_test

In [ ]:
df_breed_test['mean']=df_breed_test.mean(axis = 1)

In [ ]:
df_breed_test['pred'] = np.where(df_breed_test['mean']>1.3,2,np.where(df_breed_test['mean']>0.7,1,0))
df_breed_test['pred']

In [ ]:
y_pet_test = []
for train_index, val_index in sss.split(X_pet, y_pet):
    print("TRAIN:", train_index, "val:", val_index)
    X_train, X_val = X_pet.loc[train_index,:], X_pet.loc[val_index,:]
    y_train, y_val = y_pet.loc[train_index,], y_pet.loc[val_index,]
    clf1 = lgb.LGBMClassifier(num_leaves = 31,learning_rate = 0.01,
                         n_estimators = 3000,objective = 'multiclass', num_class = 3,
                         reg_lambda = 3, silent = False).fit(X_train,np.array(y_train))
    y_preds = clf1.predict(X_val)
    y_pet_temp = clf1.predict(X_test_pet)
    y_pet_test.append(y_pet_temp)
    f1 = accuracy_score(y_val,y_preds)
    print(f1)
print(y_pet_test)

In [ ]:
lgb.plot_importance(clf1,figsize = (20,20))

In [ ]:
df_pet_test = pd.DataFrame(y_pet_test)
df_pet_test = df_pet_test.T
df_pet_test['mean']= df_pet_test.mean(axis = 1)
df_pet_test['pred'] = np.where(df_pet_test['mean']>2.4,4,np.where(df_pet_test['mean']>1.3,2,np.where(df_pet_test['mean']>0.7,1,0)))
df_pet_test

In [ ]:
def sub_file():
    sub_df = pd.concat([df_breed_test['pred'],df_pet_test['pred']],axis = 1).reset_index()
    sub_df.columns = ['pet_id','breed_category','pet_category']
    sub_df['pet_id'] = test['pet_id']
    sub_df['breed_category'] = sub_df['breed_category'].astype('int64')
    sub = sub_df.to_csv('sub.csv',index = False)
    print(sub_df)
    return sub
sub = sub_file()